In [ ]:
import pickle
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import time
from pupilsense.inference.inference_pupilsense import Inference, get_center_and_radius
import time
from datetime import datetime

In [2]:
input_file_location = '/ceph/mrsic_flogel/public/projects/AtApSuKuSaRe_20250129_HFScohort2/TAA0000066/ses-022_date-20250509_protocol-t16/behav/Camera2_2025-05-09T14_27_37.mp4'

In [3]:
# Set up params
input_file_location = Path(input_file_location)

saving_file_location = Path(input_file_location).parent / 'pupil_extracted'
saving_file_location.mkdir(parents=False, exist_ok=True)
_saving_file_location_predicted = saving_file_location / 'predicted'
_saving_file_location_predicted.mkdir(parents=False, exist_ok=True)

PSInference = Inference(config_path=None, model_path=None)

PupilSense Inference class is initialized!
Using device: cuda
Config file: /nfs/nhome/live/skuroda/Workstation2025/PupilSense/models/Config/config.yaml
Model file: /nfs/nhome/live/skuroda/Workstation2025/PupilSense/models/model_final.pth


/nfs/nhome/live/skuroda/.conda/envs/pupilsense2/lib/python3.11/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load

In [4]:
# Load eye video
print('Loading eye video')
eye_video = cv2.VideoCapture()
eye_video.open(str(input_file_location))

Loading eye video


True

In [5]:
i_frame = 1000

In [6]:
eye_video.set(cv2.CAP_PROP_POS_FRAMES, i_frame)
_, frame = eye_video.read()
frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

# run prediction on video frame
output = PSInference.predict(frame)
instances = output["instances"]
boxes = instances.pred_boxes.tensor.cpu().numpy()

/nfs/nhome/live/skuroda/.conda/envs/pupilsense2/lib/python3.11/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [12]:
classes = instances.pred_classes
scores = instances.scores

In [13]:
instances_with_scores = [(i, score) for i, score in enumerate(scores)]
instances_with_scores.sort(key=lambda x: x[1], reverse=True)

In [21]:
index = instances_with_scores[0][0]

In [26]:
pupil = boxes[index]

In [28]:
pupil_info = get_center_and_radius(pupil)

In [ ]:
print('Loading output pickle file')
file_path = "pupil_sense_output.p"

with open(file_path, "rb") as f:
    data = pickle.load(f)